<a href="https://colab.research.google.com/github/a151815058/clv_pred/blob/main/clvInitData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import Data

In [ ]:
customerOrderDT = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/佐登妮絲/青年計劃/clv_pred/Online Retail.xlsx')
customerOrderDT

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
customerOrderDT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
customerOrderDT[customerOrderDT['Description'].isna()].index
customerOrderDT[customerOrderDT['CustomerID'].isna()].index

Int64Index([   622,   1443,   1444,   1445,   1446,   1447,   1448,   1449,
              1450,   1451,
            ...
            541531, 541532, 541533, 541534, 541535, 541536, 541537, 541538,
            541539, 541540],
           dtype='int64', length=135080)

In [ ]:
customerOrderDT.drop(customerOrderDT[customerOrderDT['Description'].isna()].index,inplace=True)
customerOrderDT.drop(customerOrderDT[customerOrderDT['CustomerID'].isna()].index,inplace=True)

In [ ]:
customerOrderDT['Profit'] = customerOrderDT['Quantity'] * customerOrderDT['UnitPrice']
customerOrderDT['InvoiceDate'] = pd.to_datetime(customerOrderDT['InvoiceDate'])
str2 = customerOrderDT['Description'].str.split(" ")
customerOrderDT['ItemSubcategory'] = str_[0] + " " + str_[1]


In [ ]:
customerOrderDT

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Profit,ItemSubcategory
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,DOORMAT MULTICOLOUR
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,DOORMAT MULTICOLOUR
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,DOORMAT MULTICOLOUR
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,DOORMAT MULTICOLOUR
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,DOORMAT MULTICOLOUR
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,DOORMAT MULTICOLOUR


In [ ]:
customerOrderDT.isna().any()

InvoiceNo          False
StockCode          False
Description        False
Quantity           False
InvoiceDate        False
UnitPrice          False
CustomerID         False
Country            False
Profit             False
ItemSubcategory    False
dtype: bool

In [ ]:
customerTotalProfitAggDT = customerOrderDT.groupby(['CustomerID'])['Profit'].sum().reset_index()
customerTotalProfitAggDT.head(5)

,CustomerID,Profit
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [ ]:
excludeCustomerVec = customerTotalProfitAggDT.loc[(customerTotalProfitAggDT['Profit'] >= customerTotalProfitAggDT['Profit'].quantile(0.8))| 
                             (customerTotalProfitAggDT['Profit'] <= customerTotalProfitAggDT['Profit'].quantile(0.01))]
excludeCustomerVec['CustomerID']

0       12346.0
1       12347.0
9       12356.0
10      12357.0
12      12359.0
         ...   
4352    18259.0
4353    18260.0
4358    18268.0
4361    18272.0
4370    18283.0
Name: CustomerID, Length: 925, dtype: float64

In [ ]:
customerOrderDT = customerOrderDT.loc[(customerOrderDT['Profit'] <= customerOrderDT['Profit'].quantile(0.8)) & 
                                      (customerOrderDT['Profit'] >= customerOrderDT['Profit'].quantile(0.01)) &
                                      customerOrderDT['CustomerID'].isin(excludeCustomerVec['CustomerID'])
                                      ]

In [ ]:
allDatesDT = pd.DataFrame(pd.date_range(start=customerOrderDT.InvoiceDate.min(),end=customerOrderDT.InvoiceDate.max()),columns={'InvoiceDate'})
allDatesDT

,InvoiceDate
0,2010-12-01 08:26:00
1,2010-12-02 08:26:00
2,2010-12-03 08:26:00
3,2010-12-04 08:26:00
4,2010-12-05 08:26:00
...,...
369,2011-12-05 08:26:00
370,2011-12-06 08:26:00
371,2011-12-07 08:26:00
372,2011-12-08 08:26:00


In [ ]:
#get the max purchase date for each customer and create a dataframe with it
tx_min_orderDT = customerOrderDT.InvoiceDate.min()
tx_min_orderDT

Timestamp('2010-12-01 08:26:00')

In [ ]:
customerOrderDT['WeekID'] = (customerOrderDT['InvoiceDate'] - tx_min_orderDT).dt.days/7
customerOrderDT['DayID'] = (customerOrderDT['InvoiceDate'] - tx_min_orderDT).dt.days
customerOrderDT

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Profit,ItemSubcategory,WeekID,DayID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,DOORMAT MULTICOLOUR,0.000000,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.000000,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.000000,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.000000,0
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30,DOORMAT MULTICOLOUR,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
541884,581585,84946,ANTIQUE SILVER T-LIGHT GLASS,12,2011-12-09 12:31:00,1.25,15804.0,United Kingdom,15.00,DOORMAT MULTICOLOUR,53.285714,373
541885,581585,21684,SMALL MEDINA STAMPED METAL BOWL,12,2011-12-09 12:31:00,0.85,15804.0,United Kingdom,10.20,DOORMAT MULTICOLOUR,53.285714,373
541886,581585,22398,MAGNETS PACK OF 4 SWALLOWS,12,2011-12-09 12:31:00,0.39,15804.0,United Kingdom,4.68,DOORMAT MULTICOLOUR,53.285714,373
541887,581585,23328,SET 6 SCHOOL MILK BOTTLES IN CRATE,4,2011-12-09 12:31:00,3.75,15804.0,United Kingdom,15.00,DOORMAT MULTICOLOUR,53.285714,373


In [ ]:
customerOrderDT = pd.merge(customerOrderDT,allDatesDT,how="outer")

In [ ]:
customerOrderDT

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Profit,ItemSubcategory,WeekID,DayID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,DOORMAT MULTICOLOUR,0.0,0.0
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.0,0.0
2,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.0,0.0
3,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,DOORMAT MULTICOLOUR,0.0,0.0
4,536365,22752,SET 7 BABUSHKA NESTING BOXES,2.0,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30,DOORMAT MULTICOLOUR,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
181877,NaN,NaN,NaN,NaN,2011-12-05 08:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181878,NaN,NaN,NaN,NaN,2011-12-06 08:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181879,NaN,NaN,NaN,NaN,2011-12-07 08:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181880,NaN,NaN,NaN,NaN,2011-12-08 08:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
customerOrderDT['CustomerID'].fillna(value=0, inplace=True)
customerOrderDT['InvoiceNo'].fillna(value=0, inplace=True)
customerOrderDT['StockCode'].fillna(value=0, inplace=True)
customerOrderDT['Description'].fillna(value=0, inplace=True)
customerOrderDT['Quantity'].fillna(value=0, inplace=True)
customerOrderDT['UnitPrice'].fillna(value=0, inplace=True)
customerOrderDT['Country'].fillna(value=0, inplace=True)
customerOrderDT['Profit'].fillna(value=0, inplace=True)
customerOrderDT['ItemSubcategory'].fillna(value=0, inplace=True)

In [ ]:
customerOrderDT.to_csv('/content/drive/MyDrive/Colab Notebooks/佐登妮絲/青年計劃/clv_pred/customerOrderDT_.csv')